In [ ]:
#Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

#Import requests library
import requests

#Use citipy module to determine city based on latitude and longitude
from citipy import citipy

#Import datetime module from datetime library
from datetime import datetime

#Import API key
from config import weather_api_key

In [ ]:
#Create a set of random latitude and longitude combinations
lat = np.random.uniform(low=-90.000, high=90.000, size=2000)
lng = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lng = zip(lat, lng)
lat_lng

In [ ]:
#Add latitudes and longitudes to new list
coords = list(lat_lng)

In [ ]:
#Review latitude and longitude combinations
for coord in coords:
    print(citipy.nearest_city(coord[0], coord[1]).city_name,
          citipy.nearest_city(coord[0], coord[1]).country_code)

In [ ]:
#Create list for cities
cities_list = []

#Locate nearest city for each latitude and longitude combination
for coord in coords:
    city = citipy.nearest_city(coord[0], coord[1]).city_name

    #If city is unique, then we will add it to the cities list
    if city not in cities_list:
        cities_list.append(city)

#Print the city count
len(cities_list)

In [ ]:
#Develop URL for Weather Map API Call
map_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

#Print map_url to verify
map_url

In [ ]:
#Create new list and dictionary to hold the weather data
city_data = []
city_weather_data = {}

#Print beginning of logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

#Create and set counters
city_record_count = 1
city_set_count = 1

#Loop through all cities in list
for i, city in enumerate(cities_list):

    #Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        city_set_count += 1
        city_record_count = 1
        
    #Create endpoint URL with each city
    city_url = map_url + "&q=" + city.replace(" ","+")

    #Log URL, record, set numbers, and city
    print(f"Processing Record {city_record_count} of Set {city_set_count} | {city}")
    
    #Add 1 to the record count
    city_record_count += 1

#Run API request for each city
    try:
        #Parse JSON and retrieve data   
        city_weather_data = requests.get(city_url).json()
        
        #Parse out needed data
        city_lat = city_weather_data["coord"]["lat"]
        city_lng = city_weather_data["coord"]["lon"]
        city_max_temp = city_weather_data["main"]["temp_max"]
        city_descr = city_weather_data["weather"][0]["description"]
        city_humidity = city_weather_data["main"]["humidity"]
        city_clouds = city_weather_data["clouds"]["all"]
        city_wind = city_weather_data["wind"]["speed"]
        city_country = city_weather_data["sys"]["country"]
       
        #Convert date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather_data["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        #Append city information into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Description": city_descr,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

#If error occurs, skip city
    except:
        print("City not found. Skipping...")
        pass

# Indicate Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
#Convert array of dictionaries to DataFrame
city_data_df = pd.DataFrame(city_data)

#Preview DataFrame
city_data_df.head(10)

In [ ]:
#Reorder columns in DataFrame
city_data_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Description', 'Max Temp', 
                       'Humidity', 'Cloudiness', 'Wind Speed']
#Reassign reordered DataFrame to city_data DataFrame
city_data_df = city_data_df[city_data_order]

#Preview reordered DataFrame
city_data_df.head(10)

In [ ]:
#Create the output file (CSV)
output_file = "WeatherPy_Database_test.csv"

In [ ]:
#Export the City_Data into CSV
city_data_df.to_csv(output_file, index_label="City_ID")